In [1]:
import importlib
from numpy.core.numeric import Inf
import torch
from torch import nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
import numpy as np
from NNet.utils.writeNNet import writeNNet
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.patches as patches
import time
import importlib
import models
import datasets
import math
import utils
from utils import *
import tasks

In [27]:
def to_device(data, device):
    return tuple(tensor.to(device) for tensor in data) if isinstance(data, tuple) else data.to(device)
def test(model, data, batch_size, compute_loss, verbose=False, compute_error=False):
    dataloader = DataLoader(data, batch_size=batch_size)
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    test_loss, correct = 0, 0
    digit_size, digit_correct = 0, 0
    positive = 0
    with torch.no_grad():
        for X, y in dataloader:
            X = to_device(X, "cuda")
            y = to_device(y, "cuda")
            if compute_error:
                acc_cnt, loss = compute_loss(model, X, y)
                test_loss += loss.cpu().item() if torch.is_tensor(loss) else loss
                correct += acc_cnt.cpu().item() if torch.is_tensor(acc_cnt) else acc_cnt
    if not compute_error:
        return None, None
    test_loss /= num_batches
    acc = correct * 1. / size
    if verbose:
        print(f"Test Error: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {test_loss:>8f} \n")

    return acc, test_loss
    
def accuracy_comparison(task_class, time_out, verified):

    training_data = datasets.LinnosDataset("../data/linnos/linnos.csv", test=False)
    print(training_data)
    task = task_class(add_counterexample = verified, incremental_training = verified, batch_counterexample = verified, early_rejection = verified, time_out=time_out) # wo finetune
    task.load_model("../model/"+task.save_name+".pth")

    print(task.save_name)
    print("data size: ", len(training_data))
    param_size = 0
    for param in task.model.parameters():
        param_size += param.nelement() * param.element_size()
    print('model size: {:.3f}KB'.format(param_size / 1024))
    def eval_loss(model, X, y):
        # normal_X, counter_X = X
        # normal_y, counter_y = y
        normal_X = X
        normal_y = y
        normal_acc_cnt, normal_loss = task.compute_normal_loss(model, normal_X, normal_y)
        return normal_acc_cnt, normal_loss

    task.model = task.model.to("cuda")
    task.model.eval()

    ### latency ###
    batch_size=1
    start = time.time()
    acc, loss = test(task.model, training_data, batch_size, eval_loss, compute_error=False)
    end = time.time()
    print("latency time (us): {:.5f}".format((end - start)/len(training_data)*1e6))
    # print("accuracy: {:.9f}".format(acc))
    # print("l2 error: {:.9f}".format(loss))
    
    ### throughput ###
    start = time.time()
    rep_times = 100
    batch_size = len(training_data)
    for t in range(rep_times):
        acc, loss = test(task.model, training_data, batch_size, eval_loss, compute_error=False)
    end = time.time()
    print("throughput: {:.5f}".format(len(training_data)/((end - start)/rep_times)))

    acc, loss = test(task.model, training_data, batch_size, eval_loss, compute_error=True)
    print("accuracy: {:.9f}".format(acc))
    print("l2 error: {:.9f}".format(loss))
    
accuracy_comparison(tasks.LinnosTask, 300, False)

LinnosTask00000-1_300
data size:  47895
model size: 365.629KB
latency time (us): 88.77328
throughput: 158423.67159
accuracy: 0.998475833
l2 error: 0.000139140


In [30]:
accuracy_comparison(tasks.LinnosTask, 300, True)

LinnosTask11110-1_300
data size:  47895
model size: 365.629KB
latency time (us): 76.53733
throughput: 170820.04905
accuracy: 0.998308801
l2 error: 0.000080312


In [60]:
def train(model, data, batch_size, compute_loss, verbose=False):
    device = "cuda"
    dataloader = DataLoader(data, batch_size=batch_size, shuffle=True)
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    train_loss, correct = 0, 0
    model = model.to(device)
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
    for batch, (X, y) in enumerate(dataloader):
        X = to_device(X, device)
        y = to_device(y, device)
        acc_cnt, loss = compute_loss(model, X, y)
        # print(acc_cnt, "/", len(X))
        correct += acc_cnt.cpu().item() if torch.is_tensor(acc_cnt) else acc_cnt
        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        train_loss += loss.cpu().item() if torch.is_tensor(loss) else loss
        # if verbose and batch % 10 == 0:
        #     loss, current = loss.item(), batch * len(X)
        #     print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
    
    train_loss /= num_batches
    acc = correct * 1. / size
    if verbose:
        print(f"Train Error: \n Accuracy: {(100*acc):>0.1f}%, Avg loss: {train_loss:>8f} \n")
    return acc, train_loss

def baseline():
    model = models.FC(1, 9, 256, 2)
    loss_fn = nn.CrossEntropyLoss()
    def compute_loss(model, x, y):
        y = y.squeeze(dim=1)
        pred = model(x)
        # print(pred)
        # print(y)
        # print(pred.argmax(1))
        acc_cnt = (pred.argmax(1) == y).type(torch.float).sum().item()
        # print(acc_cnt)
        # assert False
        loss = loss_fn(pred, y)
        return acc_cnt, loss

    training_data = datasets.LinnosDataset("../data/linnos/linnos.csv", test=False)
    print(np.shape(training_data.xs))
    print(np.max(training_data.ys))
    print(np.min(training_data.ys))
    print(np.mean(training_data.ys))
    print(np.median(training_data.ys))
    threshold = np.median(training_data.ys)
    training_data.ys = training_data.ys > threshold
    training_data.ys = training_data.ys.astype(int)
    for ep in range(10000):
        print("epoch: ", ep)
        acc, train_loss = train(model, training_data, 5000, compute_loss, verbose=True)

baseline()

(47895, 9)
1.0
0.0
0.00701667
0.00058624975
Train Error: 
 Accuracy: 65.1%, Avg loss: 0.676724 

Train Error: 
 Accuracy: 66.8%, Avg loss: 0.651355 

Train Error: 
 Accuracy: 66.8%, Avg loss: 0.628649 

Train Error: 
 Accuracy: 66.8%, Avg loss: 0.609756 

Train Error: 
 Accuracy: 66.9%, Avg loss: 0.594513 

Train Error: 
 Accuracy: 66.8%, Avg loss: 0.583791 

Train Error: 
 Accuracy: 66.8%, Avg loss: 0.577042 

Train Error: 
 Accuracy: 66.9%, Avg loss: 0.572903 

Train Error: 
 Accuracy: 66.9%, Avg loss: 0.570310 

Train Error: 
 Accuracy: 67.0%, Avg loss: 0.568888 

Train Error: 
 Accuracy: 67.0%, Avg loss: 0.567373 

Train Error: 
 Accuracy: 67.1%, Avg loss: 0.565448 

Train Error: 
 Accuracy: 67.1%, Avg loss: 0.564221 

Train Error: 
 Accuracy: 67.1%, Avg loss: 0.562913 

Train Error: 
 Accuracy: 67.2%, Avg loss: 0.561739 

Train Error: 
 Accuracy: 67.3%, Avg loss: 0.560606 

Train Error: 
 Accuracy: 67.4%, Avg loss: 0.558941 

Train Error: 
 Accuracy: 67.4%, Avg loss: 0.558599 

Tr